In [1]:
from IPython.display import Audio
import numpy as np
import scipy.io.wavfile
import pandas as pd


from scipy.fftpack import fft
from scipy import signal
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from glob import glob
from os.path import join, abspath
from os import listdir, getcwd

import librosa
from audiolazy import *
%matplotlib inline

## MCC

In [16]:
dir_path = 'data/'
full_path = join(abspath(getcwd()),dir_path, "*.wav")

n_mfcc=20
n_data=len(glob(full_path))
ukuran=np.zeros(shape=(n_data,n_mfcc))
#ukuran=np.empty(shape=[n_data,n_mfcc])
ukuran=pd.DataFrame(ukuran)
for file_name in glob(full_path):
    sample_rate, samples = scipy.io.wavfile.read(file_name)
    #print sample_rate
    samples=samples[0:int(3.5 * sample_rate)]
    
    S = librosa.feature.melspectrogram(samples, sr=sample_rate, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=n_mfcc)
    for j in range(n_mfcc):
        for i in range(n_mfcc):
            #print np.mean(mfcc[j][i])
            ukuran[j][i]=np.mean(mfcc[j][i])

ukuran['class']=None
ukuran['class'][:2]='OK'
ukuran['class'][2:4]='No'
ukuran

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,class
0,-795.432218,35.970238,-14.833815,28.815805,-14.721906,19.036209,5.313501,-35.151647,18.840924,-12.992666,...,28.161885,11.770693,-11.489983,-1.369369,-1.984072,9.970393,4.283526,-16.480585,-2.714869,OK
1,-814.875798,40.621469,-5.892730,19.256439,-11.342942,11.233098,2.637898,-27.200744,15.790923,-7.561739,...,20.706727,6.931099,-11.994173,-1.529951,-7.573341,4.184928,2.319815,-15.768163,-2.221081,OK
2,-852.077196,50.657204,12.393577,2.044998,4.737755,-3.952396,-11.259450,-3.961865,2.754834,-2.441120,...,-0.500637,3.492281,-2.607305,-7.972123,-5.364679,-2.198916,-3.158557,-4.289786,-3.611337,No
3,-847.212185,59.087908,21.051456,5.326199,0.842524,-11.764843,-18.949588,-9.338782,0.071845,-2.784443,...,-4.204624,-1.599677,-2.957116,-4.343794,-3.779825,-2.291657,-0.514011,-0.101787,-2.374789,No
4,-843.667844,61.742911,20.260771,3.522590,-0.747694,-11.665991,-15.886241,-6.715947,-2.189572,-8.579350,...,-5.857458,-1.240237,-2.747688,-4.855525,-3.592778,-1.793900,-2.024740,-2.532650,-1.937836,None


### JANGAN LUPA GAMBAR SPEKTRUM PER VOICE

In [17]:
pre_emphasis = 0.97
for file_name in glob(full_path):
    sample_rate, samples = scipy.io.wavfile.read(file_name)
    #print sample_rate
    samples=samples[0:int(3.5 * sample_rate)]
    emphasized_signal = np.append(samples[0], samples[1:] - pre_emphasis * samples[:-1])
    
    print emphasized_signal

[ -1.   -21.03  -7.66 ...,  10.01   3.34  -0.54]
[ 1153.      79.59    41.94 ...,  -136.94   -57.34  -104.28]
[ 1153.      79.59    41.94 ...,  -136.94   -57.34  -104.28]
[ 1153.      79.59    41.94 ...,  -136.94   -57.34  -104.28]
[ 1153.      79.59    41.94 ...,  -136.94   -57.34  -104.28]


#### JIKA MENGAMBIL SATU SUARA 

In [25]:
frame_size = 0.025
frame_stride = 0.01

frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
signal_length = len(emphasized_signal)
frame_length = int(round(frame_length))
frame_step = int(round(frame_step))
num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

pad_signal_length = num_frames * frame_step + frame_length
z = np.zeros((pad_signal_length - signal_length))
pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
frames = pad_signal[indices.astype(np.int32, copy=False)]
print frames
print samples

[[  1.15300000e+03   7.95900000e+01   4.19400000e+01 ...,  -7.27700000e+01
   -8.22700000e+01  -4.20100000e+01]
 [ -6.56700000e+01   1.88100000e+01   2.99000000e+01 ...,   6.87000000e+01
   -1.83200000e+01  -1.42010000e+02]
 [ -3.91400000e+01  -7.96500000e+01   1.26400000e+01 ...,  -4.61500000e+01
    1.79300000e+01  -1.15010000e+02]
 ..., 
 [ -3.43600000e+01  -1.72000000e+01  -8.50000000e+00 ...,  -7.80500000e+01
   -5.34600000e+01  -4.40900000e+01]
 [ -5.27400000e+01  -1.14850000e+02  -1.33880000e+02 ...,   2.26700000e+01
   -1.77300000e+01   6.40000000e-01]
 [ -6.79400000e+01   1.81900000e+01  -1.80700000e+01 ...,   9.16300000e+01
    5.60700000e+01   3.93100000e+01]]
[ 1153  1198  1204 ..., -1978 -1976 -2021]


In [27]:
frames *= np.hamming(frame_length)
print len(frames)

348


In [29]:
NFFT = 512
mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitude of the FFT
pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum

print mag_frames
print pow_frames

[[  2.36371525e+03   1.88606715e+03   2.04603096e+03 ...,   1.41161804e+02
    7.60530471e+01   1.46778981e+01]
 [  2.55187999e+03   2.12438165e+03   1.04336007e+03 ...,   1.25775396e+02
    2.47734946e+02   1.94409125e+02]
 [  2.28151709e+03   2.92930735e+03   4.23113419e+03 ...,   3.55166119e+02
    3.92406292e+02   4.28075754e+02]
 ..., 
 [  1.89143269e+04   1.21750321e+04   5.25091944e+03 ...,   2.77052084e+02
    2.77205857e+02   2.61912241e+02]
 [  9.59301684e+03   4.63939729e+03   1.81750748e+03 ...,   2.65544477e+02
    2.24012685e+02   1.99426817e+02]
 [  1.54334653e+03   1.24045216e+03   8.28063274e+01 ...,   2.46156759e+02
    3.96156707e+02   3.95135586e+02]]
[[  1.09124019e+04   6.94775253e+03   8.17625526e+03 ...,   3.89192480e+01
    1.12970039e+01   4.20782602e-01]
 [  1.27189287e+04   8.81444805e+03   2.12617235e+03 ...,   3.08973636e+01
    1.19868367e+02   7.38181795e+01]
 [  1.01666411e+04   1.67594561e+04   3.49658135e+04 ...,   2.46372993e+02
    3.00747457e+02   

## FILTER BANK 

In [43]:
nfilt = 40

low_freq_mel = 0
high_freq_mel = (2595 * np.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
bin = np.floor((NFFT + 1) * hz_points / sample_rate)

fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
for m in range(1, nfilt + 1):
    f_m_minus = int(bin[m - 1])   # left
    f_m = int(bin[m])             # center
    f_m_plus = int(bin[m + 1])    # right

    for k in range(f_m_minus, f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
    for k in range(f_m, f_m_plus):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
filter_banks = np.dot(pow_frames, fbank.T)
filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
filter_banks = 20 * np.log10(filter_banks)

tb=[]
for i in range (20):#jika mengikuti dengan mcc
    tb.append(np.mean(filter_banks[i]))
print tb
pd.DataFrame(tb).T

[73.018004373663857, 69.075593351359458, 73.609446134996844, 74.446726844832796, 78.199612332993453, 76.990398776287662, 74.125777500295925, 78.238411223548752, 78.692659596020846, 78.307115791441944, 76.698374326910582, 76.541919617557141, 79.116862956955543, 78.170641376593537, 79.510822305029464, 81.760304470588693, 76.468310505002151, 78.792269450519896, 82.296684227820947, 80.961582787496255]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,73.018004,69.075593,73.609446,74.446727,78.199612,76.990399,74.125778,78.238411,78.69266,78.307116,76.698374,76.54192,79.116863,78.170641,79.510822,81.760304,76.468311,78.792269,82.296684,80.961583


## LPC 

## FFT

In [15]:
from scipy import fftpack
for file_name in glob(full_path):
    sample_rate, samples = scipy.io.wavfile.read(file_name) # masih belum selesai
    sample_freq=fftpack.fftfreq(sample_rate)
    print samples

[ -1 -22 -29 ..., -22 -18 -18]
[ 1153  1198  1204 ..., -2248 -2157 -2233]
[ 1153  1198  1204 ..., -2248 -2157 -2233]
[ 1153  1198  1204 ..., -2248 -2157 -2233]
[ 1153  1198  1204 ..., -2248 -2157 -2233]


In [ ]:
from scipy.fft import 

## STATISTIC 

- High Frequency Energy
- F0 Mean
- F0 range
- Speech and articulation rate
- Prosody
- Formants
- HNR

## Classification 

In [115]:
from sklearn.svm import SVC
from sklearn import metrics
clf=SVC() #Default hyperparameters
clf.fit(ukuran.iloc[:4,:-1],ukuran['class'][:4])
y_pred=clf.predict(ukuran.iloc[4:,:-1])
print('Accuracy Score:')
#print(metrics.accuracy_score(y_test,y_pred))
y_pred

Accuracy Score:


array(['No'], dtype=object)

In [ ]:
# main()